In [1]:
!pip install finance-datareader
!pip install opendartreader
!pip install --upgrade opendartreader

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 주식가격 로딩
import FinanceDataReader as fdr

# 재무재표 로딩
import OpenDartReader

import datetime

In [4]:
def pro_equ(code, start, end):
    # OpenDartReader 사용
    api_key = '9b9157d4313a84f3141d73f879c5d239d49bb0ce'
    dart = OpenDartReader(api_key)
    arr = [] # 재무 정보
    index = [] # 계산지표
    
    # 주식정보
    if type(code) != str:
        code = str(code).zfill(6)       
    
    if type(start) != str:
        start = str(start)
    
    if type(end) != str:
        end = str(end)
        
    stock = fdr.DataReader(code, start = str(int(start)+1), end = end)  # 2021 지표 <- 2020,2019 데이터로 만듬
    stock = stock.reset_index()
    stock['PER'] = 0.1
    stock['PBR'] = 0.1
    stock['ROE'] = 0.1
    stock['ROA'] = 0.1
    
    #         ------------------------
    
    small = dart.report('005930', '소액주주', int(datetime.datetime.now().strftime("%Y"))-1, reprt_code=11014)
    stock_tot_co = int(small['stock_tot_co'].str.replace(',', '')) # 총 발행 주식 수
    
    for i in range(int(start[:4]), int(end[:4])):
        # https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS003&apiId=2019016 --> 입력변수 명 참고
        # corp: 종목코드, bsns_year: 사업연도, reprt_code: 보고서코드
        # (1분기보고서 : 11013, 반기보고서 : 11012, 3분기보고서 : 11014, 사업보고서 : 11011)
        fs_before = dart.finstate_all(corp = code, bsns_year= str(i-1), fs_div='CFS', reprt_code=11011) 
        fs_present = dart.finstate_all(corp = code, bsns_year= i, fs_div='CFS', reprt_code=11014)

        # 자본, 부채는 당기 금액('thstrm_amount') 값을 통해서 구함
#         equity = int(fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Equity']), 'thstrm_amount'].replace(",", "")) # 당기자본(자본총계)
        equity = int(fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_nm'].isin(['자본총계']), 'thstrm_amount'])
    
#         liability = int(fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Liabilities']), 'thstrm_amount'].replace(",", "")) # 당기부채(부채총계)
        liability = int(fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_nm'].isin(['부채총계']), 'thstrm_amount'].replace(",", ""))
        # 자산 총계
        assets = equity + liability
        
        # <예시>
        # 2019 4분기 ~ 2020 3분기까지의 당기순이익의 합을 구하려면 2019년 4분기 당기순이익과 2020년 1분기 ~ 3분기 당기순이익의 합을 알아여함
        # 2020년 1분기 ~ 3분기 당기순이익의 합은 2020년 3분기 손익계산서에서 'thstrm_add_amount' 값을 가져오면 되고
        # 2019년 4분기 당기순이익은 2019년 전체 당기순이익에서 2019년 1분기 ~ 3분기 당기순이익의 합을 빼서 구할 수 있음
        
        # frmtrm_add_amount --> 전기 금액, thstrm_amount ---> 당기 금액, thstrm_add_amount --> 당기 누적금액
#         profit_before_3Q = int(fs_present.loc[fs_present['sj_div'].isin(['IS']) & fs_present['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'frmtrm_add_amount'].replace(",", "")) # 당기순이익
        profit_before_3Q = int(fs_present.loc[fs_present['sj_div'].isin(['IS']) & fs_present['account_nm'].isin(['지배기업의 소유주에게 귀속되는 당기순이익(손실)']), 'frmtrm_add_amount'].replace(",", ""))
        
#         profit_before = int(fs_before.loc[fs_before['sj_div'].isin(['IS']) & fs_before['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'thstrm_amount'].replace(",", "")) # 당기순이익
        profit_before = int(fs_before.loc[fs_before['sj_div'].isin(['IS']) & fs_before['account_nm'].isin(['지배기업의 소유주에게 귀속되는 당기순이익(손실)']), 'thstrm_amount'].replace(",", ""))
        
#         profit_present = int(fs_present.loc[fs_present['sj_div'].isin(['IS']) & fs_present['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'thstrm_add_amount'].replace(",", "")) # 당기순이익
        profit_present = int(fs_present.loc[fs_present['sj_div'].isin(['IS']) & fs_present['account_nm'].isin(['지배기업의 소유주에게 귀속되는 당기순이익(손실)']), 'thstrm_add_amount'].replace(",", "")) # 당기순이익
        
        profit = (profit_before-profit_before_3Q) + profit_present
        
        a = (equity, liability, assets, profit, stock_tot_co) 
        # 자본총계(당기자본), 부채총계(당기부채), 자산총계, 당기 순이익, 총 발행 주식수
        arr.append(a)
        
#         - PER : 주가를 1주당 순이익(EPS)로 나눈 값
#         - PBR : 주가를 1주당 순자산가치로 나눈 값
#         - ROE : 당기순이익을 순자산(자기자본)으로 나눈 값
#         - ROA : 당기순이익을 자산총액으로 나눈 값
        
#         print(stock[stock['Date'].dt.year == i]['Close'])
            
        
        ROA = profit/assets
        stock.loc[stock['Date'].dt.year == i+1,'ROA'] = ROA
        
        EPS = profit/stock_tot_co
        PER = stock.loc[stock['Date'].dt.year == i+1,'Close']/EPS
        
#         PER = stock[stock['Date'].dt.year == i+1]['Close']/EPS
        
        BPS = equity/stock_tot_co
        PBR = stock.loc[stock['Date'].dt.year == i+1, 'Close']/BPS
#         PBR = stock[stock['Date'].dt.year == i+1]['Close']/BPS
        
        for i in PER.index:
            stock['PER'][i] = PER[i]
            stock['PBR'][i] = PBR[i]
        
        b = (EPS,BPS)
        # EPS,BPS
        
        index.append(b)
        
    stock['ROE'] = stock['PBR']/stock['PER']
    
    return arr,index, stock

In [5]:
end_date = datetime.datetime.now().strftime("%Y%m%d")

In [10]:
arr, indexs, df = pro_equ(5930,2017,end_date)  # 2018의 경우 2019,2020,2021까지의 결과 도출

In [11]:
df

,Date,Open,High,Low,Close,Volume,Change,PER,PBR,ROE,ROA
0,2018-01-02,51380,51400,50780,51020,169485,0.001177,8.403220,1.445614,0.172031,0.122212
1,2018-01-03,52540,52560,51420,51620,200270,0.011760,8.502043,1.462615,0.172031,0.122212
2,2018-01-04,52120,52180,50640,51080,233909,-0.010461,8.413103,1.447314,0.172031,0.122212
3,2018-01-05,51300,52120,51200,52120,189623,0.020360,8.584395,1.476782,0.172031,0.122212
4,2018-01-08,52400,52520,51500,52020,167673,-0.001919,8.567925,1.473949,0.172031,0.122212
...,...,...,...,...,...,...,...,...,...,...,...
907,2021-09-06,76800,77600,76600,77300,12861180,0.009138,18.552538,1.671147,0.090076,0.066190
908,2021-09-07,77100,77100,75900,76100,13239401,-0.015524,18.264530,1.645204,0.090076,0.066190
909,2021-09-08,76000,76400,75600,76300,11798147,0.002628,18.312532,1.649528,0.090076,0.066190
910,2021-09-09,76400,76600,75000,75300,17600770,-0.013106,18.072525,1.627909,0.090076,0.066190


In [14]:
arr

[(210691251000000,
  85887328000000,
  296578579000000,
  36245426000000,
  5969782550),
 (242103160000000,
  95092626000000,
  337195786000000,
  47577132000000,
  5969782550),
 (263442244000000,
  89943741000000,
  353385985000000,
  24607128000000,
  5969782550),
 (276136188000000,
  99652554000000,
  375788742000000,
  24873372000000,
  5969782550)]

In [13]:
indexs

[(6071.481782866614, 35292.952337099785),
 (7969.659129376496, 40554.77029058621),
 (4121.947121842822, 44129.28641764347),
 (4166.54573121763, 46255.65264517047)]

### ============= 연습 =============

In [270]:
stock.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 912 entries, 2018-01-02 to 2021-09-10
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    912 non-null    int64  
 1   High    912 non-null    int64  
 2   Low     912 non-null    int64  
 3   Close   912 non-null    int64  
 4   Volume  912 non-null    int64  
 5   Change  912 non-null    float64
dtypes: float64(1), int64(5)
memory usage: 49.9 KB


In [445]:
stock = stock.reset_index()

In [351]:
pd.merge(stock,PER)

,Date,Open,High,Low,Close,Volume,Change
0,2020-12-09,72100,73900,72000,73900,24077996,0.030683
1,2020-12-23,72400,74000,72300,73900,19411326,0.022130
2,2021-08-18,73900,74600,73100,73900,29192631,-0.004043
3,2020-12-17,73400,73700,72600,73300,24293214,-0.006775
4,2021-08-23,73300,74000,73000,73300,19384648,0.008253
...,...,...,...,...,...,...,...
479,2021-09-03,76400,76700,76000,76600,12096419,0.007895
480,2021-09-06,76800,77600,76600,77300,12861180,0.009138
481,2021-09-07,77100,77100,75900,76100,13239401,-0.015524
482,2021-09-08,76000,76400,75600,76300,11798147,0.002628


In [367]:
PER = stock[stock['Date'].dt.year == 2021]['Close']/10
for i in PER.index:
    stock['PER'][i] = PER[i]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [440]:
stock['per'] = 0.1

In [431]:
a,b = 15,10

In [448]:
stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668 entries, 0 to 667
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    668 non-null    datetime64[ns]
 1   Open    668 non-null    int64         
 2   High    668 non-null    int64         
 3   Low     668 non-null    int64         
 4   Close   668 non-null    int64         
 5   Volume  668 non-null    int64         
 6   Change  668 non-null    float64       
 7   per     668 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(5)
memory usage: 41.9 KB


In [456]:
stock.loc[stock['Date'].dt.year == 2020,'per'] = b

In [457]:
stock

,Date,Open,High,Low,Close,Volume,Change,per
0,2019-01-02,39400,39400,38550,38750,7847664,0.001292,0.1
1,2019-01-03,38300,38550,37450,37600,12471493,-0.029677,0.1
2,2019-01-04,37450,37600,36850,37450,14108958,-0.003989,0.1
3,2019-01-07,38000,38900,37800,38750,12748997,0.034713,0.1
4,2019-01-08,38000,39200,37950,38100,12756554,-0.016774,0.1
...,...,...,...,...,...,...,...,...
663,2021-09-06,76800,77600,76600,77300,12861180,0.009138,15.0
664,2021-09-07,77100,77100,75900,76100,13239401,-0.015524,15.0
665,2021-09-08,76000,76400,75600,76300,11798147,0.002628,15.0
666,2021-09-09,76400,76600,75000,75300,17600770,-0.013106,15.0


In [458]:
stock[stock['Date'].dt.year == 2020]

,Date,Open,High,Low,Close,Volume,Change,per
246,2020-01-02,55500,56000,55000,55200,12993228,-0.010753,10.0
247,2020-01-03,56000,56600,54900,55500,15422255,0.005435,10.0
248,2020-01-06,54900,55600,54600,55500,10278951,0.000000,10.0
249,2020-01-07,55700,56400,55600,55800,10009778,0.005405,10.0
250,2020-01-08,56200,57400,55900,56800,23501171,0.017921,10.0
...,...,...,...,...,...,...,...,...
489,2020-12-23,72400,74000,72300,73900,19411326,0.022130,10.0
490,2020-12-24,74100,78800,74000,77800,32502870,0.052774,10.0
491,2020-12-28,79000,80100,78200,78700,40085044,0.011568,10.0
492,2020-12-29,78800,78900,77300,78300,30339449,-0.005083,10.0


In [363]:
PER[738]

8300.0

In [364]:
stock['PER'][738]

83000.0

In [365]:
stock['PER'][738] = PER[738]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [366]:
stock['PER'][738]

8300.0

In [343]:
PER.index

738

In [321]:
stock[stock['Date'].dt.year == 2021]['PER'] = PER

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [322]:
stock[stock['Date'].dt.year == 2021]['PER']

738   NaN
739   NaN
740   NaN
741   NaN
742   NaN
       ..
907   NaN
908   NaN
909   NaN
910   NaN
911   NaN
Name: PER, Length: 174, dtype: float64

In [313]:
stock

,Date,Open,High,Low,Close,Volume,Change,PER
0,2018-01-02,51380,51400,50780,51020,169485,0.001177,51020.0
1,2018-01-03,52540,52560,51420,51620,200270,0.011760,51620.0
2,2018-01-04,52120,52180,50640,51080,233909,-0.010461,51080.0
3,2018-01-05,51300,52120,51200,52120,189623,0.020360,52120.0
4,2018-01-08,52400,52520,51500,52020,167673,-0.001919,52020.0
...,...,...,...,...,...,...,...,...
907,2021-09-06,76800,77600,76600,77300,12861180,0.009138,NaN
908,2021-09-07,77100,77100,75900,76100,13239401,-0.015524,NaN
909,2021-09-08,76000,76400,75600,76300,11798147,0.002628,NaN
910,2021-09-09,76400,76600,75000,75300,17600770,-0.013106,NaN


In [227]:
a =stock.copy()
a=a.reset_index()

In [290]:
s = a[a['Date'].dt.year == 2018]['Close']

In [292]:
a['aaaa'] = 0.1

In [295]:
a.head()

,Date,Open,High,Low,Close,Volume,Change,aaaa
0,2018-01-02,51380,51400,50780,51020,169485,0.001177,51020.0
1,2018-01-03,52540,52560,51420,51620,200270,0.011760,51620.0
2,2018-01-04,52120,52180,50640,51080,233909,-0.010461,51080.0
3,2018-01-05,51300,52120,51200,52120,189623,0.020360,52120.0
4,2018-01-08,52400,52520,51500,52020,167673,-0.001919,52020.0


In [294]:
a['aaaa'] = s

In [247]:
a.loc[a['Date'].dt.year == 2018,'Close']/1

0      51020.0
1      51620.0
2      51080.0
3      52120.0
4      52020.0
        ...   
239    38650.0
240    38800.0
241    38350.0
242    38250.0
243    38700.0
Name: Close, Length: 244, dtype: float64

In [ ]:
a.loc[a['sj_div'].isin(['BS']) & a['account_nm'].isin(['부채총계']),'thstrm_amount']

In [183]:
dart.report('005930', '소액주주', 2020, reprt_code=11014)

,rcept_no,corp_cls,corp_code,corp_name,se,shrholdr_co,shrholdr_tot_co,shrholdr_rate,hold_stock_co,stock_tot_co,hold_stock_rate
0,20201116001248,Y,00126380,삼성전자,소액주주,"1,754,623","1,754,776",99.99%,"3,711,273,219","5,969,782,550",62.17%


In [ ]:
fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Liabilities']), 'thstrm_amount']

In [ ]:
int(fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_nm'].isin(['부채총계']), 'thstrm_amount'].replace(",", ""))

In [ ]:
int(fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Liabilities']), 'thstrm_amount'].replace(",", ""))

In [87]:
fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_nm'].isin(['자본총계']), 'thstrm_amount']

52    275948016000000
Name: thstrm_amount, dtype: object

=============코드 수정=======오류 파악============

In [6]:
api_key = '9b9157d4313a84f3141d73f879c5d239d49bb0ce'
dart = OpenDartReader(api_key)

In [121]:
fs_before = dart.finstate_all(corp = '005930', bsns_year= '2020', fs_div='CFS', reprt_code=11011) 
fs_present = dart.finstate_all(corp = '005930', bsns_year= '2020',fs_div='CFS', reprt_code=11014)

In [122]:
fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Equity']), 'thstrm_amount']

52    276136188000000
Name: thstrm_amount, dtype: object

In [123]:
fs_present.iloc[52]

rcept_no               20201116001248
reprt_code                      11014
bsns_year                        2020
corp_code                    00126380
sj_div                             BS
sj_nm                           재무상태표
account_id           ifrs-full_Equity
account_nm                       자본총계
account_detail                      -
thstrm_nm                 제 52 기 3분기말
thstrm_amount         276136188000000
frmtrm_nm                     제 51 기말
frmtrm_amount         262880421000000
ord                                55
thstrm_add_amount                 NaN
frmtrm_q_nm                       NaN
frmtrm_q_amount                   NaN
frmtrm_add_amount                 NaN
Name: 52, dtype: object

In [124]:
fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Liabilities']), 'thstrm_amount']

42    99652554000000
Name: thstrm_amount, dtype: object

In [125]:
fs_present.iloc[42]

rcept_no                    20201116001248
reprt_code                           11014
bsns_year                             2020
corp_code                         00126380
sj_div                                  BS
sj_nm                                재무상태표
account_id           ifrs-full_Liabilities
account_nm                            부채총계
account_detail                           -
thstrm_nm                      제 52 기 3분기말
thstrm_amount               99652554000000
frmtrm_nm                          제 51 기말
frmtrm_amount               89684076000000
ord                                     44
thstrm_add_amount                      NaN
frmtrm_q_nm                            NaN
frmtrm_q_amount                        NaN
frmtrm_add_amount                      NaN
Name: 42, dtype: object

In [73]:
a = dart.finstate_all(corp = '005930', bsns_year= '2019')

In [77]:
a.loc[a['sj_div'].isin(['BS']) & a['account_nm'].isin(['부채총계']),'thstrm_amount']

44    89684076000000
Name: thstrm_amount, dtype: object

In [ ]:
frmtrm_add_amount

In [98]:
fs_before.loc[fs_before['sj_div'].isin(['IS']) & fs_before['account_nm'].isin(['지배기업의 소유주에게 귀속되는 당기순이익(손실)']), 'thstrm_amount'].replace(",", "")

68    26090846000000
Name: thstrm_amount, dtype: object

In [95]:
fs_before.loc[fs_before['sj_div'].isin(['IS']) & fs_before['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'thstrm_amount'].replace(",", "")

68    26090846000000
Name: thstrm_amount, dtype: object

In [96]:
fs_before.iloc[68]

rcept_no                                               20210309000744
reprt_code                                                      11011
bsns_year                                                        2020
corp_code                                                    00126380
sj_div                                                             IS
sj_nm                                                           손익계산서
account_id           ifrs-full_ProfitLossAttributableToOwnersOfParent
account_nm                                 지배기업의 소유주에게 귀속되는 당기순이익(손실)
account_detail                                                      -
thstrm_nm                                                      제 52 기
thstrm_amount                                          26090846000000
frmtrm_nm                                                      제 51 기
frmtrm_amount                                          21505054000000
bfefrmtrm_nm                                                   제 50 기
bfefrmtrm_amount    

In [126]:
a = dart.finstate_all(corp = '005930', bsns_year= '2018',fs_div='CFS', reprt_code=11011)

In [127]:
a.loc[a['sj_div'].isin(['IS']) & a['account_nm'].isin(['지배기업의 소유주에게 귀속되는 당기순이익(손실)']), 'thstrm_amount'].replace(",", "")

73    43890877000000
Name: thstrm_amount, dtype: object

In [128]:
a = dart.finstate_all(corp = '005930', bsns_year= '2020',fs_div='CFS', reprt_code=11011)

In [129]:
fs_present.columns

Index(['rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'sj_div', 'sj_nm',
       'account_id', 'account_nm', 'account_detail', 'thstrm_nm',
       'thstrm_amount', 'frmtrm_nm', 'frmtrm_amount', 'ord',
       'thstrm_add_amount', 'frmtrm_q_nm', 'frmtrm_q_amount',
       'frmtrm_add_amount'],
      dtype='object')

In [131]:
fs_present.loc[fs_present['sj_div'].isin(['IS']) & fs_present['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'frmtrm_add_amount'].replace(",", "") # 당기순이익

68    16277059000000
Name: frmtrm_add_amount, dtype: object

In [132]:
fs_present.iloc[68]

rcept_no                                               20201116001248
reprt_code                                                      11014
bsns_year                                                        2020
corp_code                                                    00126380
sj_div                                                             IS
sj_nm                                                           손익계산서
account_id           ifrs-full_ProfitLossAttributableToOwnersOfParent
account_nm                                 지배기업의 소유주에게 귀속되는 당기순이익(손실)
account_detail                                                      -
thstrm_nm                                                  제 52 기 3분기
thstrm_amount                                           9266814000000
frmtrm_nm                                                         NaN
frmtrm_amount                                                     NaN
ord                                                                15
thstrm_add_amount   

In [134]:
fs_present.loc[fs_present['sj_div'].isin(['IS']) & fs_present['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'thstrm_add_amount'].replace(",", "")

68    19645377000000
Name: thstrm_add_amount, dtype: object

In [39]:
fs_present['sj_div'].isin(['BS']).index

RangeIndex(start=0, stop=205, step=1)

In [37]:
fs_present['account_id'].isin(['기말자본']).index

RangeIndex(start=0, stop=205, step=1)

In [80]:
int(fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Liabilities']), 'thstrm_amount'].replace(",", ""))

102287702000000

In [45]:
fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Equity'])]

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,ord,thstrm_add_amount,frmtrm_q_nm,frmtrm_q_amount,frmtrm_add_amount
52,20201116001248,11014,2020,00126380,BS,재무상태표,ifrs-full_Equity,자본총계,-,제 52 기 3분기말,276136188000000,제 51 기말,262880421000000,55,NaN,NaN,NaN,NaN


In [47]:
fs_2019 = dart.finstate_all(corp='005930', bsns_year='2019', fs_div='CFS', reprt_code=11011) 
fs_2020_3Q = dart.finstate_all(corp='005930', bsns_year='2020', fs_div='CFS', reprt_code=11014) 



In [50]:
int(fs_2020_3Q.loc[fs_2020_3Q['sj_div'].isin(['IS']) & fs_2020_3Q['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'frmtrm_add_amount'].replace(",", ""))


16277059000000

In [49]:
int(fs_2020_3Q.loc[fs_2020_3Q['sj_div'].isin(['IS']) & fs_2020_3Q['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'thstrm_add_amount'].replace(",", ""))


19645377000000

In [41]:
fs_present['account_id'].isin(['ifrs-full_Equity'])

0      False
1      False
2      False
3      False
4      False
       ...  
200    False
201    False
202    False
203    False
204    False
Name: account_id, Length: 205, dtype: bool